# Fourier space

In [1]:
import numpy as np
import torch
from scipy.special import erfc

def dx( x ):
    return x[None, :, :] - x[:, None, :]

def distance( x ):
    return torch.norm(x[None, :, :] - x[:, None, :], dim=-1)

def qpairs( q ):
    return q[None, :]*q[:, None]

In [2]:
def nk( maxki ):
    """Combinatorics, 
    returns number of combis for lattice vectors in 3D
    """
    mitnull = (maxki + 1) ** 3 * 2 ** 3
    korr = 7 + sum([30 + i * 24 for i in range(maxki)])
    
    return mitnull - korr

def kvec( maxki=3, maxnk=300, maxr=0):
    """Returns k-vectors for k-part of ewald
    
    Arguments:
        maxki (int): max value for kx, ky, kz
        maxnk (int): max # of k vectors, unrestrict with maxnk=0 
        maxr  (int): max length of k, unrestrict with maxr=0
    
    Output:
        k (float): k vectors (dim = 3 x maxnk)
    """
    vector = np.zeros([nk(maxki), 3], dtype=np.float32)
    cnt = 0
    for i in range(maxki + 1):
        for j in range(maxki + 1):
            for k in range(maxki + 1):
                if (maxr == 0 or maxr >= i**2+j**2+k**2):
                    vector[cnt] = [i, j, k]
                    cnt += 1
                    if (i > 0):
                        vector[cnt] = [-i, j, k]
                        cnt += 1
                    if (j > 0):
                        vector[cnt] = [i, -j, k]
                        cnt += 1
                    if (k > 0):
                        vector[cnt] = [i, j, -k]
                        cnt += 1
                    if (i > 0 and j > 0):
                        vector[cnt] = [-i, -j, k]
                        cnt += 1
                    if (i > 0 and k > 0):
                        vector[cnt] = [-i, j, -k]
                        cnt += 1
                    if (j > 0 and k > 0):
                        vector[cnt] = [i, -j ,-k]
                        cnt += 1
                    if (i > 0 and j > 0 and k > 0):
                        vector[cnt] = [-i, -j, -k]
                        cnt += 1
    if (maxnk > 0 ):   
        indexlist = np.argsort(np.linalg.norm(vector,axis=-1))
        vector = torch.from_numpy(vector[indexlist])
        vector = vector[0:maxnk]
    else:
        vector = torch.from_numpy(vector[0:cnt])
    return vector

In [3]:
k = kvec() # default k: 300 vectors
print(len(k), k[-1])
k = kvec(10, 0) # unlimited number of k vectors
print(len(k), k[-1])
k = kvec(5, 1) # specific number of k vectors
print(len(k), k[-1])
k = kvec(5, 0, 15) # max len(k)
print(len(k), k[-1])

300 tensor([-1.,  3., -3.])
9261 tensor([-10., -10., -10.])
1 tensor([0., 0., 0.])
251 tensor([-3., -2., -1.])


In [4]:
def kvec_time(ki):
    import time
    t1 = time.perf_counter()
    k = kvec(ki, 0)
    t2 = time.perf_counter()
    return ki, len(k), float(torch.norm(k[-1])), t2-t1

for i in range(9):
    print(kvec_time(i**2))

(0, 1, 0.0, 4.513999738264829e-05)
(1, 27, 1.7320507764816284, 0.00011525800073286518)
(4, 729, 6.928203105926514, 0.0012855839959229343)
(9, 6859, 15.588457107543945, 0.007277222000993788)
(16, 35937, 27.712812423706055, 0.045372453998425044)
(25, 132651, 43.30126953125, 0.13022325999918394)
(36, 389017, 62.35382843017578, 0.38159373399685137)
(49, 970299, 84.87049102783203, 0.9323252000031061)
(64, 2146689, 110.85124969482422, 2.161930767004378)
(81, 4330747, 140.29611206054688, 4.779811213003995)


# Structure Factor squared

$$\mathrm{sfac} = |S(\textbf{k})|^2 = 
\sum_{a,b}^N q_a q_b \cos(<\textbf{k}, r_{ab}>)$$

- saved as vector ( dim(sfactor) = dim(k) )

In [5]:
def sfac( x, q, k ):
    """Structure factor squared
    """
    return torch.sum(
        torch.sum( qpairs( q ) * np.cos( 
        torch.sum( k[:, None, None] * dx( x ), 
                  dim=-1 )), dim=-1), dim=-1)

# K-space
$$ E_L = \frac{1}{2V\varepsilon_0} 
    \sum_\textbf{k} |S(\textbf{k})|^2 \frac{\exp
    \left(-\sigma^2 k^2\right)}{k^2}$$
    
$$ \mathrm{with} \quad |\textbf{k}| = k $$

In [6]:
def kwald( x, q, lbox=1, kmax=3 ):
    """Returns the fourier space part of energy of ewald summation
    
    Arguments:
        x    (float): position vectors (dim = n x 3)
        q      (int): charges (dim = n)
        lbox (float): box length
        kmax   (int): max value for k_i \in {k_x, k_y, k_z}
        
        n := # of particles
        sigma (float): sqrt(variance), width of gaussian
    
    Output:
        e_l   (float): long range part of ewald potential
    """
    sigma = lbox/10.
    eps = 1 # 8.854187817e-12
    
    k = kvec(kmax, 0)*(2*np.pi/lbox)
    s = sfac(x, q, k)
    
    ksq = torch.sum(k**2, dim=-1)
    kinv = ksq.clone()
    kinv[kinv != 0] = 1 / kinv[kinv != 0]
    frac = np.exp(-sigma ** 2 * ksq / 2) * kinv
    
    return torch.sum(s * frac) / (2 * lbox ** 3 * eps)

In [7]:
pos = torch.tensor([[0., 0, 3],
        [0, 1, 3],
        [1, 1, -3],
        [1, 0, -3]])
cha = torch.tensor([-1., 2, -1, 3])

In [8]:
import time

n = 50 # number of particles
position = torch.from_numpy(
    np.random.rand(n, 3).astype(dtype=np.float32))
charge = torch.from_numpy(
    np.random.choice([-1.,1], n).astype(dtype=np.float32))

for i in range(1, 6):
    t1 = time.perf_counter()
    e_l = float(kwald(position, charge, 1, i**2))
    t2 = time.perf_counter()
    print(i**2, e_l, t2-t1)

1 7.805880546569824 0.03231446800054982
4 11.193275451660156 0.04279659200255992
9 11.209467887878418 0.492462517999229
16 11.209465980529785 2.4706974270011415
25 11.209467887878418 8.213287564998609
